In [1]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-06-23 19:21:29--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-23 19:21:30 (12.4 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [3]:
!wget -q https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

In [48]:
import minsearch
import json


from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('api_key')

In [3]:
with open('documents.json', 'r') as fp:
    docs_raw = json.load(fp)

In [15]:
document = []
for docs in docs_raw:
   for doc in docs['documents']:
       doc['course'] = docs['course']
       document.append(doc)

In [17]:
document[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [19]:
mi = minsearch.Index(text_fields = ['question', 'text', 'section'], keyword_fields=['course'])

In [20]:
q = 'the course has already started, can I still enroll?'

In [22]:
mi.fit(document)

In [55]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5, 'text': 1.0}
    filter_dict = {'course': 'data-engineering-zoomcamp'}
    
    result = mi.search(query, filter_dict=filter_dict, boost_dict=boost)
    return result

In [92]:
def build_prompt(query, search_result):
    prompt_template = '''
    You're a course teaching assistant. Answer the QUESTION base on the CONTEXT from the FAQ database.
    use only the facts from the CONTEXT when answering the QUESTION.
    if the CONTEXT doesn't contain the answer, output NONE.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    
    '''.strip()
    context = ''

    for doc in search_result: 
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [93]:
def llm(prompt):
    model = "mistral-small-latest"
    
    client = MistralClient(api_key=api_key)
    
    chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content=prompt)]
    )


    return chat_response.choices[0].message.content

In [100]:
def rag(query):
    search_result = search(query)
    prompt = build_prompt(query, search_result)
    return llm(prompt)

In [103]:
print(rag('how do I get the certificate for the course?'))

To get the certificate for the course, you need to finish the course with a “live” cohort. You need to peer-review capstone(s) after submitting a project, which can only be done when the course is running. Certificates are not awarded for the self-paced mode.
